<a href="https://colab.research.google.com/github/Yashu2699/Deep_learning/blob/main/GAN/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from __future__ import print_function, division

from keras.datasets import fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np


In [4]:
(training_data, _), (_, _) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
x_train = training_data / 127.5 - 1
x_train = np.expand_dims(x_train, axis=3)

In [8]:
def build_generator():
  model = Sequential()

  model.add(Dense(128*7*7, activation='relu', input_dim=latent_dim))
  model.add(Reshape((7,7,128)))
  model.add(UpSampling2D())

  model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization(momentum=0.8))
  model.add(UpSampling2D())

  model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Conv2D(channels, kernel_size=3, activation='tanh', padding='same'))

  model.summary()

  noise = Input(shape=(latent_dim, ))
  img = model(noise)

  return Model(noise, img)

In [9]:
def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape= img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape= img_shape)
    validity = model(img)

    return Model(img, validity)

In [10]:

img_shape = (28, 28, 1)
channels = 1
latent_dim = 100

optimizer = Adam(0.0002, 0.5)

#build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#build the generator
generator = build_generator()

z = Input(shape=(latent_dim, ))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 zero_padding2d (ZeroPadding  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 64)         256       
 ormalization)                                          

In [11]:
def train(epochs, batch_size=128, save_interval=50):
  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    #train discriminator
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5* np.add(d_loss_real, d_loss_fake)

    #train generator
    g_loss = combined.train_on_batch(noise, valid)

    print("%d [D loss: %f, acc: %.2f] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if(epoch % save_interval == 0):
      plot_generated_images(epoch, generator)

In [12]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
train(epochs=1000, batch_size=32, save_interval=1)